# Tutorial 6: Robustness

In this tutorial we explore what different climate change scenarios and different adaptation options might mean for Conowingo Dam management objectives, and for the different actors who pursue these objectives.

As usual, we start with library imports.

In [ ]:
from model import setup, balance_calcs, performance, visuals
import numpy as np
import pandas as pd
import copy
import os

# Part 1 - Analysis setup

## 1.1 - Problem framing

Our XLRM is as follows:

**X**

>* Flow variability is represented by the historical time series.
>* We also account for climate change: we are going to simulate what would happen in a drier future. To not deal with several variables at a time, here we use the uniform model to reduce flows by increments of 5%, up to 30%.

**L**

> We assume a standard operating policy (SOP) for the day-to-day operational release decisions of the reservoir.

To plan for climate change, we want to explore the following four options:
> L0: current setup;
> 
> L1: a new Peach Bottom intake at the same height as the Baltimore intake;
> 
> L2: a new desalination plant at Baltimore, that replaces the withdrawal from Conowingo when the reservoir is less than full (i.e. Baltimore withdrawals drop to zero then);
> 
> L3: the two options above implemented simultaneously.

**R**

> In previous tutorials, we built a water balance model of the system, and are going to use it here.

**M**

*For each management objective we will compute **R-R-V indicators, volumetric reliability, and count of the number of failure events** (note in this tutorial we count failure events per year, rather than across all 70 years of simulation).* These objectives are:
> 1. Meeting the residential demand for the city of Baltimore;
> 2. Meeting the residential demand for the city of Chester;
> 3. Meeting the industrial demand for the Peach Bottom nuclear plant;
> 4. Respecting the environmental flow requirement (ecological uses);
> 5. Avoiding flows above 15,000 m3/s that lead to flooding downstream in the town of Port Deposit;
> 6. Hydropower production. Contrary to the other, this objective is only evaluated by computing the average annual hydropower production.


## 1.2 - Setting up the model (R)

As usual we start with preparing the Conowingo system model.

In [ ]:
# Preparing the model
reservoir_name = 'Conowingo'
downstream_demand_names = ['Environmental']
direct_demand_names = ['Baltimore', 'Chester', 'Nuclear plant']

# Loading the model!
conowingo_default = setup.define_reservoir(reservoir_name, downstream_demand_names, direct_demand_names)

# Read flow and demand data. We keep this copy of the data for the simulation of different futures.
flows_default = setup.extract_flows(reservoir=conowingo_default)

In [ ]:
results_dir = 'results/robustness/'
os.makedirs(results_dir, exist_ok = True)

# Part 2 - Exploring futures and options

**The water balance will be performed for each of the 7 climate change scenarios we consider in X, and for each of the 4 adaptation options we consider in L.**


In [ ]:
# Let us declare the number of futures and options
nb_futures = 7
nb_levers = 4

# Let us also declare the number of years of simulation
nb_years = 70

For this exploration, we will do a double loop on climate scenarios and on levers, and we will need to store the results for robustness analysis. First, we will need to initialise these arrays.

>* RRV-type indicators and failure count, each in a 7 by 4 by 5 array (third dimension storing the performance indicators of the 5 objectives that are not hydropower);

In [ ]:
# R-R-V metrics
reliability = np.zeros((nb_futures, nb_levers, 5))
resilience = np.zeros((nb_futures, nb_levers, 5))
vulnerability = np.zeros((nb_futures, nb_levers, 5))

# Annual failure rate
failure_rate = np.zeros((nb_futures, nb_levers, 5))

>* Volumetric reliability in a 7 by 4 by 4 array, since volumetric reliability only concerns objectives 1 to 4;

In [ ]:
volumetric_reliability = np.zeros((nb_futures, nb_levers, 4))

>* Average annual hydropower production in a 7 by 4 array.

In [ ]:
hp_annual = np.zeros((nb_futures, nb_levers))

Once this preliminary work is complete, we have our main double loop. For each inflow scenario (loop on sc) and for each lever combination (loop on l) we compute performance and store it. Note you can toggle `robustness_simulations` from 1 to 0 once you have simulated results, to not have to recompute results every time you run the analysis.

In [ ]:
robustness_simulations = 1

if robustness_simulations == 1:

    # Loop on flow scenarios
    for sc in range(nb_futures):

        # Loop on levers
        for l in range(nb_levers):

            # Re-initialise "flows" and get right value of inflows
            flows = flows_default.copy()
            flows['Total inflows (m3/s)'] = flows['Total inflows (m3/s)'] * (1-0.05 * sc)

            # Re-initialise "reservoir"
            reservoir = copy.deepcopy(conowingo_default)

            # Lowering the nuclear plant intake to the level of Baltimore's (for L1 and L3, i.e., when L is an odd number)
            if l % 2 == 1:
                reservoir.demand_on_site[2].intake_depth = reservoir.demand_on_site[0].intake_depth

            # Action plans involving cutting Conowingo dam supply to Baltimore when the reservoir is not full.
            # This is equivalent to having the intake at full reservoir level, and happens for L2 and L3
            if l > 1:
                reservoir.demand_on_site[0].intake_depth = 0.001  # 0 would result in 0 reliability

            # Now we simulate the water balance given inflows and levers
            flows = balance_calcs.sop_full(reservoir, flows)

            # We measure performance
            metrics = performance.all_metrics(reservoir=reservoir, water_balance=flows)
            # Change vulnerability from string ('XX.XX%') to numeric
            metrics['Vulnerability'] = pd.to_numeric(metrics['Vulnerability'].str.replace('%', ''), errors='coerce')

            # Storing results
            # R-R-V metrics
            reliability[sc, l, :] = metrics.loc[:, 'Reliability (0-1)']
            resilience[sc, l, :] = metrics.loc[:, 'Resilience (-)']
            vulnerability[sc, l, :] = metrics.loc[:, 'Vulnerability']
            # Failure rate
            failure_rate[sc, l, :] = metrics.loc[:, 'Failure count'] / nb_years
            # Volumetric reliability
            volumetric_reliability[sc, l, 0:4] = metrics.loc[0:3, 'Volumetric reliability']
            # Hydropower (divide by 1,000 to convert from MWh to GWh)
            hp_annual[sc, l] = reservoir.daily_production(flows).sum() / nb_years / 1000

    # Save results
    np.save(results_dir + 'reliability.npy', reliability)
    np.save(results_dir + 'resilience.npy', resilience)
    np.save(results_dir + 'vulnerability.npy', vulnerability)
    np.save(results_dir + 'failure_rate.npy', failure_rate)
    np.save(results_dir + 'volumetric_reliability.npy', volumetric_reliability)
    np.save(results_dir + 'hp_annual.npy', hp_annual)

else:

    # We need to load results for what follows.
    reliability = np.load(results_dir + 'reliability.npy')
    resilience = np.load(results_dir + 'resilience.npy')
    vulnerability = np.load(results_dir + 'vulnerability.npy')
    failure_rate = np.load(results_dir + 'failure_rate.npy')
    volumetric_reliability = np.load(results_dir + 'volumetric_reliability.npy')
    hp_annual = np.load(results_dir + 'hp_annual.npy')                

# Part 3 - Regret analysis

In this part, we are now analysing the results by measuring how actions and the deviations between scenarios interact to improve or degrade performance. We call these measures *regret measures*. Here we consider two types of regret measures:

> 1. Deviation from the performance in the baseline scenario for each lever L. This enables us to understand if a lever leads to very different performance depending on what version of the future happens, i.e., if actioning a lever makes the system more or less robust.
> 2. Deviation from the best lever in each climate scenario. This enables us to directly compare decisions in each scenario.


In [ ]:
lever_list = ['L0', 'L1', 'L2', 'L3']
scenario_list = np.arange(0, 35, 5)

## 3.1 Hydropower results

Let us start with the simpler metrics, hydropower.

In [ ]:
fig = visuals.plot_regret(array=hp_annual, 
                          metric_name='Annual hydropower production (GWh)', 
                          ref_dim=1, 
                          lever_list=lever_list, 
                          scenario_list=scenario_list,
                          scenario_label='Flow reduction (%)')

>* **Question 1. What do we learn from plotting regret as a deviation from the most favourable scenario for hydropower?**

In [ ]:
fig = visuals.plot_regret(array=hp_annual, 
                          metric_name='Annual hydropower production (GWh)', 
                          ref_dim=0, 
                          lever_list=lever_list, 
                          scenario_list=scenario_list,
                          scenario_label='Flow reduction (%)')

>* **Question 2. What do we learn from plotting regret as a deviation from the most favourable lever for hydropower?**


## 3.2 Multi-actor analysis

First, pick your indicator of choice, and the associated array of results. 
Higher values must be better, so put a minus if needed -- as in `- vulnerability`.
Also pick your actor: (0) for Baltimore, (1) for Chester, (2) for the nuclear plant, (3) for the environment (and (4) for flooding if you're curious).

In [ ]:
my_metric = 'Reliability'
results = reliability
actor = 2
actor_names = ['Baltimore', 'Chester', 'Nuclear plant', 'Environmental flows', 'Flooding']

>* **Question 3. Given the XLRM given at the beginning of the tutorial, does reliability mean the same thing for Baltimore and other actors?**

Let us plot both measures of regret for the chose metric and actor.

In [ ]:
fig = visuals.plot_regret(array=results[:, :, actor], 
                          metric_name=my_metric + ', ' + actor_names[actor], 
                          ref_dim=1, 
                          lever_list=lever_list, 
                          scenario_list=scenario_list,
                          scenario_label='Flow reduction (%)')

In [ ]:
fig = visuals.plot_regret(array=results[:, :, actor], 
                          metric_name=my_metric + ', ' + actor_names[actor], 
                          ref_dim=0, 
                          lever_list=lever_list, 
                          scenario_list=scenario_list,
                          scenario_label='Flow reduction (%)')

>* **Question 4. Using `reliability` as the metric from which to compute regret, what do we learn about the impact of climate and levers on the different actors?**
>  
>* **Question 5. If we try and do the regret analysis with `resilience`, `vulnerability`, or `event_rate` (instead of `reliability`), what do we learn?**
>  
>* **Question 6. What are the limits of this regret analysis?**